## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-25-15-06-00 +0000,wsj,Oil Futures Little Changed As Market Awaits U....,https://www.wsj.com/finance/commodities-future...
1,2026-02-25-15-03-00 +0000,wsj,David Tepper Calls for Sweeping Changes at Whi...,https://www.wsj.com/finance/investing/david-te...
2,2026-02-25-15-00-38 +0000,nypost,Speaker Johnson: I ‘probably could have ejecte...,https://nypost.com/2026/02/25/us-news/speaker-...
3,2026-02-25-14-58-28 +0000,nypost,Storage giant is latest major company to flee ...,https://nypost.com/2026/02/25/us-news/public-s...
4,2026-02-25-14-58-19 +0000,bbc,Labour MPs call for urgent action on 'unfair' ...,https://www.bbc.com/news/articles/c4g2kxjg1e1o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2562/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
132,trump,84
20,union,43
235,new,26
374,year,18
276,address,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2026-02-25-05-07-00 +0000,wsj,President Trump told a national audience in hi...,https://www.wsj.com/politics/policy/trump-hail...,233
212,2026-02-25-03-04-21 +0000,nyt,"“After four years, in which millions and milli...",https://www.nytimes.com/live/2026/02/25/us/fac...,210
171,2026-02-25-04-57-00 +0000,wsj,President Trump delivered a State of the Union...,https://www.wsj.com/politics/policy/five-takea...,185
224,2026-02-25-02-43-12 +0000,nypost,"From USA hockey team to 100-year-old veteran, ...",https://nypost.com/2026/02/24/us-news/usa-chan...,185
220,2026-02-25-02-44-20 +0000,nypost,Trump announces USA goalie Connor Hellebuyck w...,https://nypost.com/2026/02/24/us-news/trump-an...,180


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,233,2026-02-25-05-07-00 +0000,wsj,President Trump told a national audience in hi...,https://www.wsj.com/politics/policy/trump-hail...
212,210,2026-02-25-03-04-21 +0000,nyt,"“After four years, in which millions and milli...",https://www.nytimes.com/live/2026/02/25/us/fac...
174,142,2026-02-25-04-39-24 +0000,nyt,“When I last spoke in the chamber 12 months ag...,https://www.nytimes.com/live/2026/02/25/us/fac...
180,91,2026-02-25-04-06-50 +0000,nyt,“They’ve killed and maimed thousands of Americ...,https://www.nytimes.com/live/2026/02/25/us/fac...
116,61,2026-02-25-09-56-55 +0000,cbc,Any trade deal with Canada will include tariff...,https://www.cbc.ca/news/politics/trump-tariffs...
194,60,2026-02-25-03-35-45 +0000,nyt,"“With the great Big Beautiful Bill, we gave yo...",https://www.nytimes.com/live/2026/02/25/us/fac...
193,55,2026-02-25-03-35-53 +0000,nyt,“I want to stop all payments to big insurance ...,https://www.nytimes.com/live/2026/02/24/us/tru...
201,55,2026-02-25-03-24-12 +0000,nyt,“Under my just-enacted most favored nation agr...,https://www.nytimes.com/live/2026/02/25/us/fac...
315,47,2026-02-24-20-26-51 +0000,nypost,"Ozempic, Wegovy prices to drop up to 50% as No...",https://nypost.com/2026/02/24/business/ozempic...
371,42,2026-02-24-17-00-00 +0000,nypost,OpenAI asks California watchdog to probe who’s...,https://nypost.com/2026/02/24/business/openai-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
